# Using a config file

## Get parameters from config .yaml file

This is the standard use case for generating the roughness blocks.
The user inputs the parameters config file containing the necessary parameters.

These parameters account for the number of blocks being generated in each direction and also the offset and spacing applied to each row.

In [3]:
import pathlib
from cfdmod.use_cases.block_gen import GenerationParams

cfg_file = pathlib.Path("./fixtures/tests/block_gen/block_params.yaml")
cfg = GenerationParams.from_file(cfg_file)

cfg

GenerationParams(N_blocks_x=10, N_blocks_y=10, block_params=BlockParams(height=10.0, width=10.0, length=10.0), spacing_params=SpacingParams(spacing=(10.0, 10.0), line_offset=10.0, offset_direction=<OffsetDirection.x: 'x'>))

## Build block and linear patterns

In order to build all the blocks, linear patterns are applied.
The first one applies to create a row in the offset direction.
The second one applies to replicate the original row and apply an offset to odd index rows.

In [6]:
from cfdmod.use_cases.block_gen import build_single_block, linear_pattern

triangles, normals = build_single_block(cfg.block_params)

single_line_triangles, single_line_normals = linear_pattern(
    triangles,
    normals,
    direction=cfg.spacing_params.offset_direction.value,
    n_repeats=cfg.single_line_blocks,
    spacing_value=cfg.single_line_spacing,
)

full_triangles, full_normals = linear_pattern(
    single_line_triangles,
    single_line_normals,
    direction=cfg.perpendicular_direction.value,
    n_repeats=cfg.multi_line_blocks,
    spacing_value=cfg.multi_line_spacing,
    offset_value=cfg.spacing_params.line_offset,
)

print("Single block:", f"Vertices count: {triangles.shape[0] * triangles.shape[1]}", f"Triangles count: {len(triangles)}")
print("Single line", f"Vertices count: {single_line_triangles.shape[0] * single_line_triangles.shape[1]}", f"Triangles count: {len(single_line_triangles)}")
print("Replicated lines", f"Vertices count: {full_triangles.shape[0] * full_triangles.shape[1]}", f"Triangles count: {len(full_triangles)}")

Single block: Vertices count: 30 Triangles count: 10
Single line Vertices count: 300 Triangles count: 100
Replicated lines Vertices count: 3000 Triangles count: 1000


## Export generated geometry

In order to use the generated geometry, the user must export the triangles and normals, using the geometry API and the STL format (triangles, normals).

In [3]:
from cfdmod.api.geometry.STL import export_stl
import pathlib

output_path = pathlib.Path("./output/block_gen")

export_stl(output_path / "blocks.stl", full_triangles, full_normals)